In [9]:
import numpy as np
import pandas as pd
import datahugger

In [13]:
dh = datahugger.get("10.17605/OSF.IO/EWS27", "../data")
dh.tree() # Remains to be done: extracting the paths instead of hardcoded (as below):

19_09_2023_22_58_21/LMAHe[...].hdf5: 100%|██████████| 18.6M/18.6M
19_09_2023_22_58_21/U_at_bottom.h5 : 100%|██████████| 147k/147k

data/
└── 19_09_2023_22_58_21/
    ├── LMAHeureuxPorosityDiff.hdf5
    └── U_at_bottom.h5

1 directories, 2 files
README Data Code


For some reason importing only the data key with pandas doesn't work so back to h5py:

In [14]:
import h5py
U_path = "../data/19_09_2023_22_58_21/U_at_bottom.h5"
U_at_bottom_and_time = h5py.File(U_path, 'r')

In [15]:
Solution_path = '../data/19_09_2023_22_58_21/LMAHeureuxPorosityDiff.hdf5' # Also to be downloaded
# Solution_file = pd.read_hdf(Solution_path, key="data", mode="r")
import h5py
Solution = h5py.File(Solution_path, 'r')
data = Solution['data']
data.shape

(3001, 5, 200)

Check the values of U at the bottom:

In [ ]:
U_at_bottom_and_times[2995:3000, 1]

In [ ]:
import pylab as pyl
pyl.figure(figsize=(10,6))
pyl.plot(U_at_bottom_and_times[:, 0], U_at_bottom_and_times[:, 1], ".")
pyl.xlabel("Times T*")
pyl.ylabel("U")
pyl.show()

### What is it supposed to be according to Eq. 46.
The formula for K as below is used for moderate (0.05, 0.95) values of phi in Fortran and throughout in Python. So it might need changing for phi at the bottom for the calculation of U.

In [ ]:
def k_normal(phi):
    k = betasV * phi ** 3 / (1 - phi) ** 2
    k = k * (1 - np.exp(-10 * (1 - phi) / phi))
    return k

Keeping the same convention as in Fortran for easy comparing, at the expense of creating redundant variables:

In [ ]:
S = float(Solution.attrs['sedimentationrate'])
Vscale = S 
beta = float(Solution.attrs['beta']) # cm/a
P_32 = beta/Vscale
betasV = P_32

In [ ]:
phi0 = float(Solution.attrs['Phi0'])
rhow = float(Solution.attrs['rhow'])
rhoa = float(Solution.attrs['rhoa'])
CA0 = float(Solution.attrs['CA0'])
rhoc = float(Solution.attrs['rhoc'])
CC0 = float(Solution.attrs['CC0'])
rhot = float(Solution.attrs['rhot'])
rhos0 = rhoa * CA0 + rhoc * CC0 + rhot * (1 - (CA0 + CC0))
rhos0

In [ ]:
phi_bottom = data[3000,4,199]
CA = data[3000,0,199]
CC = data[3000,1,199]
rhos_bottom = rhoa * CA + rhoc * CC + rhot * (1 - (CA + CC))

In [ ]:
U = 1 - k_normal(phi0)/S * (1 - phi0) * (rhos0/rhow - 1) + k_normal(phi_bottom)/S * (1 - phi_bottom) * (rhos_bottom/rhow - 1)

In [ ]:
U

In [ ]:
import matplotlib.pyplot as plt
t = np.array(Solution['times'][:])
s = np.array(data[:,:,0])
fig, ax = plt.subplots()
ax.plot(t, s)
ax.set(xlabel='time [T*]', ylabel='Compounds/Porosity',
       title=f"Phi0 = {phi0}, PhiIni = {Solution.attrs['PhiIni']}")
ax.grid()